<a href="https://colab.research.google.com/github/hashmil/MidiGen/blob/main/MidiGen_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Initial Setup
run_on_colab = True

if(run_on_colab):
    from google.colab import drive
    drive.mount('/content/drive')

In [2]:
# Load Processed Data and n_vocab
import pickle
import numpy
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.callbacks import ModelCheckpoint

with open('/content/drive/MyDrive/MIDI/vivaldi/pickle/notes.pkl', 'rb') as filepath:
    notes = pickle.load(filepath)
with open('/content/drive/MyDrive/MIDI/vivaldi/pickle/n_vocab.pkl', 'rb') as filepath:
    n_vocab = pickle.load(filepath)

In [3]:
# Prepare Sequences
sequence_length = 100
pitchnames = sorted(set(item for item in notes))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []

for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
network_input = network_input / float(n_vocab)
network_output = to_categorical(network_output)

In [4]:
# Define Network Structure
def create_network(network_input, n_vocab):
    model = Sequential()
    model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

model = create_network(network_input, n_vocab)

In [ ]:
# Compile and Train the Model
filepath = "/content/drive/MyDrive/MIDI/vivaldi/weights/weights-{epoch:02d}-{loss:.4f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.fit(network_input, network_output, epochs=100, batch_size=64, callbacks=callbacks_list)